In [11]:
# Import necessary libraries
import os
import sys

# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))
import pandas as pd
from load_data import fetch_data
from sklearn.preprocessing import MinMaxScaler



In [5]:

# SQL query to fetch data
query = "SELECT * FROM xdr_data ;"  

# Load the data
df = fetch_data(query)

d:\Kifya_training\Week 2\Telecom-User-Analysis-and-Dashboard-Development\scripts\load_data.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [ ]:

# Aggregate the metrics
aggregated_metrics = df.groupby('MSISDN/Number').agg({
    'Dur. (ms)': 'sum',  # Total session duration per customer
    'Total DL (Bytes)': 'sum',  # Total download traffic per customer
    'Total UL (Bytes)': 'sum',  # Total upload traffic per customer
}).reset_index()



In [7]:
# Calculate session frequency
session_frequency = df['MSISDN/Number'].value_counts().reset_index()
session_frequency.columns = ['MSISDN/Number', 'Session Frequency']




In [8]:
# Merge session frequency with other metrics
aggregated_metrics = pd.merge(aggregated_metrics, session_frequency, on='MSISDN/Number')

# Calculate total traffic (Download + Upload)
aggregated_metrics['Total Traffic (Bytes)'] = aggregated_metrics['Total DL (Bytes)'] + aggregated_metrics['Total UL (Bytes)']



In [9]:
# Sort the aggregated data by each metric for ranking
top_10_duration = aggregated_metrics.sort_values(by='Dur. (ms)', ascending=False).head(10)
top_10_traffic = aggregated_metrics.sort_values(by='Total Traffic (Bytes)', ascending=False).head(10)
top_10_frequency = aggregated_metrics.sort_values(by='Session Frequency', ascending=False).head(10)



In [10]:
# Display the top 10 customers for each metric
print("Top 10 Customers by Session Duration:")
print(top_10_duration)

print("\nTop 10 Customers by Total Traffic:")
print(top_10_traffic)

print("\nTop 10 Customers by Session Frequency:")
print(top_10_frequency)

Top 10 Customers by Session Duration:
       MSISDN/Number   Dur. (ms)  Total DL (Bytes)  Total UL (Bytes)  \
13180   3.362578e+10  18553754.0      7.770043e+09       729577380.0   
6437    3.361489e+10   9966898.0      8.156743e+09       689483001.0   
92923   3.376054e+10   9279434.0      7.811295e+09       703478581.0   
13526   3.362632e+10   8791927.0      7.301517e+09       669650721.0   
65118   3.366716e+10   8744914.0      5.052068e+09       566326364.0   
50281   3.366284e+10   6614270.0      3.593674e+09       406424372.0   
57160   3.366469e+10   6288730.0      5.279827e+09       402180804.0   
666     3.360313e+10   6287761.0      4.540607e+09       435587631.0   
66029   3.366746e+10   5649882.0      2.334299e+09       266222647.0   
92577   3.376041e+10   5321667.0      6.610852e+09       521518890.0   

       Session Frequency  Total Traffic (Bytes)  
13180                 17           8.499621e+09  
6437                  17           8.846226e+09  
92923              

In [12]:


# Select the metrics for normalization
metrics_to_normalize = ['Session Frequency', 'Dur. (ms)', 'Total Traffic (Bytes)']

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the metrics
normalized_data = scaler.fit_transform(aggregated_metrics[metrics_to_normalize])

# Convert normalized data back to a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=metrics_to_normalize)

# Add the MSISDN/Number column to the normalized DataFrame for reference
normalized_df['MSISDN/Number'] = aggregated_metrics['MSISDN/Number']




In [13]:
# Display the first few rows of the normalized data
print("Normalized Data:")
print(normalized_df.head())

Normalized Data:
   Session Frequency  Dur. (ms)  Total Traffic (Bytes)  MSISDN/Number
0           0.000000   0.005908               0.095931   3.360100e+10
1           0.000000   0.009387               0.014026   3.360100e+10
2           0.000000   0.006892               0.063851   3.360100e+10
3           0.000000   0.002304               0.044148   3.360101e+10
4           0.058824   0.001615               0.161598   3.360101e+10
